In [1]:
import pandas as pd
from sqlalchemy import create_engine

import sys, os

# Helpers
abspath = os.path.abspath
dirname = os.path.dirname
sep = os.sep
file_ = os.getcwd()

sys.path.append(dirname(abspath(file_)))

import utils.folder_tb as fo
import utils.mining_data_tb as md
import utils.sql_tb as sq

## Data to upload

In [2]:
# Load the data and check that everything is fine
cleaned_data_path = fo.path_to_folder(2, "data" + sep + "7_cleaned_data") + "cleaned_data.csv"
data = pd.read_csv(cleaned_data_path)

print(data.shape)
data.info()

(13446, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13446 entries, 0 to 13445
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      13446 non-null  int64  
 1   MCQ160H   13446 non-null  int64  
 2   RIDAGEYR  13446 non-null  int64  
 3   BPXDI1    13446 non-null  float64
 4   BPXSY1    13446 non-null  float64
 5   BMXWT     13446 non-null  float64
 6   BMXWAIST  13446 non-null  float64
 7   LBXTC     13446 non-null  float64
 8   LBXSGL    13446 non-null  float64
 9   MEANCHOL  13446 non-null  float64
 10  MEANTFAT  13446 non-null  float64
 11  MEANSFAT  13446 non-null  float64
 12  MEANSUGR  13446 non-null  float64
 13  MEANFIBE  13446 non-null  float64
 14  MEANTVB6  13446 non-null  float64
 15  Female    13446 non-null  int64  
 16  Male      13446 non-null  int64  
dtypes: float64(12), int64(5)
memory usage: 1.7 MB


In [3]:
model_comparisons_path = fo.path_to_folder(2, "models")

# First model table
model_comparison_noscale_nobalance = pd.read_csv(model_comparisons_path + "model_comparison_noscale_nobalance.csv")
model_comparison_noscale_nobalance.rename(columns = {"Unnamed: 0" : "Model"}, inplace = True)

# Second model table
model_comparison_scale_balance = pd.read_csv(model_comparisons_path + "model_comparison_scale_balance.csv")
model_comparison_scale_balance.rename(columns = {"Unnamed: 0" : "Model"}, inplace = True)

In [4]:
model_comparison_noscale_nobalance

,Model,Test_score,Train_score,Test_score_drop,Accuracy,Precision,Recall,F1_score,Confusion_matrix
0,"LogisticRegression(max_iter=300, n_jobs=-1, ra...",0.921190,0.916512,0.005104,0.921190,0.580000,0.131818,0.214815,[[2449 21]\n [ 191 29]]
1,"RandomForestClassifier(max_depth=10, n_jobs=-1...",0.919331,0.934827,-0.016577,0.919331,0.800000,0.018182,0.035556,[[2469 1]\n [ 216 4]]
2,"RandomForestClassifier(max_depth=20, n_jobs=-1...",0.919331,0.971830,-0.054021,0.919331,0.571429,0.054545,0.099585,[[2461 9]\n [ 208 12]]
3,"LogisticRegression(n_jobs=-1, random_state=42)",0.918587,0.916512,0.002265,0.918587,0.510204,0.113636,0.185874,[[2446 24]\n [ 195 25]]
4,"RandomForestClassifier(n_jobs=-1, random_state...",0.918216,0.972109,-0.055439,0.918216,0.500000,0.036364,0.067797,[[2462 8]\n [ 212 8]]
5,"RandomForestClassifier(max_features='sqrt', n_...",0.918216,0.972109,-0.055439,0.918216,0.500000,0.036364,0.067797,[[2462 8]\n [ 212 8]]
6,"RandomForestClassifier(max_features='log2', n_...",0.918216,0.972109,-0.055439,0.918216,0.500000,0.036364,0.067797,[[2462 8]\n [ 212 8]]
7,KNeighborsClassifier(n_jobs=-1),0.914498,0.918929,-0.004822,0.914498,0.421875,0.122727,0.190141,[[2433 37]\n [ 193 27]]
8,"KNeighborsClassifier(n_jobs=-1, weights='dista...",0.914498,0.969319,-0.056556,0.914498,0.421875,0.122727,0.190141,[[2433 37]\n [ 193 27]]


## SQL connection

In [5]:
# Load server setting into variables
read_json = md.read_json_to_dict("sql_server_settings.json")

IP_DNS = read_json["IP_DNS"]
USER = read_json["USER"]
PASSWORD = read_json["PASSWORD"]
DB_NAME = read_json["DB_NAME"]
PORT = read_json["PORT"]

In [6]:
# Connection with the database
sql_db = sq.MySQL(IP_DNS, USER, PASSWORD, DB_NAME, PORT)
sql_db.connect()

Connected to MySQL server [45395203b_ds_april_2021_db]


In [7]:
db_connection_str = sql_db.SQL_ALCHEMY
db_connection = create_engine(db_connection_str)

In [8]:
### DON'T EXECUTE THIS ONE

# Uploading the data into the db
# Dataset
data.to_sql("jonathan_suarez_caceres", con = db_connection, index = False)

# First model comparison
model_comparison_scale_balance.to_sql("model_comparison_scale_balance", con = db_connection, index = False)

# Second model comparison
model_comparison_noscale_nobalance.to_sql("model_comparison_noscale_nobalance", con = db_connection, index = False)

In [10]:
# Close connection
sql_db.close()

Close connection with MySQL server [45395203b_ds_april_2021_db]


# Just for testing purposes

In [9]:
# Just for testing purposes

sql_query = '''
SELECT * FROM model_comparison_noscale_nobalance
'''

results = sql_db.execute_get_sql(sql_query)
column_names = [tuple[0] for tuple in sql_db.cursor.description]
pd.DataFrame(results, columns = column_names)

Executing:
 
SELECT * FROM model_comparison_noscale_nobalance



,Model,Test_score,Train_score,Test_score_drop,Accuracy,Precision,Recall,F1_score,Confusion_matrix
0,"LogisticRegression(max_iter=300, n_jobs=-1, ra...",0.921190,0.916512,0.005104,0.921190,0.580000,0.131818,0.214815,[[2449 21]\n [ 191 29]]
1,"RandomForestClassifier(max_depth=10, n_jobs=-1...",0.919331,0.934827,-0.016577,0.919331,0.800000,0.018182,0.035556,[[2469 1]\n [ 216 4]]
2,"RandomForestClassifier(max_depth=20, n_jobs=-1...",0.919331,0.971830,-0.054021,0.919331,0.571429,0.054545,0.099585,[[2461 9]\n [ 208 12]]
3,"LogisticRegression(n_jobs=-1, random_state=42)",0.918587,0.916512,0.002265,0.918587,0.510204,0.113636,0.185874,[[2446 24]\n [ 195 25]]
4,"RandomForestClassifier(n_jobs=-1, random_state...",0.918216,0.972109,-0.055439,0.918216,0.500000,0.036364,0.067797,[[2462 8]\n [ 212 8]]
5,"RandomForestClassifier(max_features='sqrt', n_...",0.918216,0.972109,-0.055439,0.918216,0.500000,0.036364,0.067797,[[2462 8]\n [ 212 8]]
6,"RandomForestClassifier(max_features='log2', n_...",0.918216,0.972109,-0.055439,0.918216,0.500000,0.036364,0.067797,[[2462 8]\n [ 212 8]]
7,KNeighborsClassifier(n_jobs=-1),0.914498,0.918929,-0.004822,0.914498,0.421875,0.122727,0.190141,[[2433 37]\n [ 193 27]]
8,"KNeighborsClassifier(n_jobs=-1, weights='dista...",0.914498,0.969319,-0.056556,0.914498,0.421875,0.122727,0.190141,[[2433 37]\n [ 193 27]]
